# 🏥 Health Inequality Analysis

## Problem Statement
Health outcomes vary significantly across countries and income groups.
Quantifying these disparities is essential for prioritizing healthcare
investment and policy intervention.

This analysis investigates the relationship between economic capacity,
health spending, and population health outcomes.

## Policy Relevance
- Supports equitable healthcare resource allocation
- Highlights regions at high health risk
- Informs long-term public health planning

## Target Variables
- Life expectancy
- Mortality indicators
- Health expenditure
- Income / economic indicators

## Scope & Limitations
- Cross-country analysis (macro-level)
- Does not capture within-country inequality
- Data reflects reported national averages


In [2]:
from utils.path_setup import setup_project_path

PROJECT_ROOT = setup_project_path()
print("Project root:", PROJECT_ROOT)
print("Path setup complete.")

Project root: d:\def_main\Code\MyProjects\eda-mlops-portfolio
Path setup complete.
